# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [62]:
from sklearn import ensemble, model_selection, metrics 

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [63]:
def write_answer(file_name, answer):
    print(answer)
    with open(file_name, "w") as f:
        f.write(answer)

In [76]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=False)

In [77]:
print(load_boston().DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Решаем задачу регрессии если что

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [84]:
# считаем вектор сдвигов по формуле для линейной регресии
def calc_s(y, y_comp):
    return y - y_comp

def gbm_predict(X, base_algorithms_list, coefficients_list):
    '''вычисляет прогноз построенной на данный момент композиции X'''
    return [sum([coeff * algo.predict([x])[0] 
                 for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

def RMSE(y, y_pred):
    return mean_squared_error(y, y_pred)**(1./2)

def boost(n_trees = 50, gen_coef = (lambda x: 0.9), max_depth = 5):
    base_algorithms_list = [] #массив для объектов DecisionTreeRegressos
    coefficients_list = []
    
    for n_tree in range(n_trees):
        s = calc_s(y_train, gbm_predict(X_train, base_algorithms_list, coefficients_list))
        new_alg = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
        new_alg.fit(X_train, s)
        new_coef = gen_coef(n_tree)
        base_algorithms_list.append(new_alg)
        coefficients_list.append(new_coef)
            
    return RMSE(y_test, gbm_predict(X_test, base_algorithms_list, coefficients_list))

In [85]:
ans2 = boost()

In [86]:
write_answer('task2.txt', str(ans2))

5.455565103009407


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [87]:
%%time 

ans3 = boost(gen_coef = lambda i: 0.9/(1.0 + i))

CPU times: user 24.4 s, sys: 83.7 ms, total: 24.5 s
Wall time: 24.5 s


In [88]:
write_answer('task3.txt', str(ans3))

4.812550945781194


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [83]:
# ToDO: тут почему-то не работал RMSE
import xgboost as xgb

# посотмрим переобучается ли градинентый бустинг с ростом числа деревьев 
n_trees = [1] + list(range(10, 55, 5))

xgb_scoring = []
for n_tree in n_trees:
    estimator = xgb.XGBClassifier(n_estimators=n_tree)
    score = model_selection.cross_val_score(estimator, X_train, y_train, 
                                             scoring = 'neg_mean_absolute_error', cv = 3)    
    xgb_scoring.append(score)
xgb_scoring = np.asmatrix(xgb_scoring)

`ntrees` variation:

In [40]:
%%time

rmse_n_tree = []
for n_trees in range(50, 76, 5):
    rmse_n_tree.append(boost(n_trees=n_trees))
    print("ntrees =", n_trees, ":",)

ntrees = 50 : 13.541414090223247
ntrees = 55 : 13.547873750269932
ntrees = 60 : 13.548054437398001
ntrees = 65 : 13.546768487540765
ntrees = 70 : 13.546897848245193
ntrees = 75 : 13.54679753825682
CPU times: user 4min 12s, sys: 1.61 s, total: 4min 14s
Wall time: 4min 16s


In [ ]:
plt.plot(rmse_n_tree)

`max_depth` variation:

In [41]:
%%time

for max_depth in range(5, 41, 5):
    print("max_depth =", max_depth, ":", boost(max_depth=max_depth))

max_depth = 5 : 13.541414090223247
max_depth = 10 : 12.4144022546694
max_depth = 15 : 12.917473292939665
max_depth = 20 : 13.563026573663564
max_depth = 25 : 13.563026573663564
max_depth = 30 : 13.563026573663564
max_depth = 35 : 13.563026573663564
max_depth = 40 : 13.563026573663564
CPU times: user 3min 18s, sys: 672 ms, total: 3min 19s
Wall time: 3min 19s


In [54]:
write_answer('task4.txt', '2 3')

2 3


## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [89]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

ans5 = RMSE(y_test, model.predict(X_test))
write_answer('task5.txt', str(ans5))

8.254979753549401
